In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !pip3 install torch
# !pip3 install torchvision

In [0]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim

In [0]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])

In [0]:
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [0]:
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In PyTorch we usually process multiple samples at the same time. Usually we stack up each training vector along rows to create $X$ matrix.
\begin{equation}
X = \begin{pmatrix}
- & - & x_1 & - & -\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
- & - & x_{batch-size} & - & -\\
\end{pmatrix}
\end{equation}

torch.nn.Linear class when instantiated creates an object with a randomly initialised $W$ matrix of dimensions(in_class features, out_class features) and another randomly initialised matrix $b$ of dimensions(out_class features, 1). These dimensions are passed as arguments to the constructor during creation of object. When the forward function of the created object is invoked with parameters $X$, it applies linear transformation $XW + b$ and returns the appropriate matrix

In [0]:
# Build a Neural Network
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(784, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 10)
        self.sigmoid = torch.nn.Sigmoid()
        self.softmax = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1.forward(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)

        return x

In [0]:
# Create an object of class MLP
model = MLP()

# Define the loss
criterion = torch.nn.CrossEntropyLoss()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr = 0.05)

In [0]:
epochs = 100
for e in range(epochs):
    print("Epoch ", (e + 1))
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        logits = model.forward(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        accuracy = 0
        with torch.no_grad():
            for images, labels in testloader:
                logit = model(images)
                ps = logit#torch.exp(logit)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        print("Loss on training set: ", running_loss)
        print("Accuracy on test set: ", end = " ")
        print((accuracy / len(testloader)).data.numpy())